In [1]:
import pandas as pd

In [2]:
df_chart_tbl = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o15_empty_chart_max_tbl.csv', index_col=False)
df_max_chart = pd.read_csv('D:\Papers\Cyprus_Paper\Data\o10_chartevent_24_hour_max.csv', index_col=False)
df_data_chart = pd.read_csv('D:\MSc Aegean\Thesis\mimic-iv-1.0\icu\d_items.csv.gz')

In [3]:
"""
Δημιουργώ εκ νέου την στήλη concat
χωρίς κενά ανάμεσα στα στοιχεία.
Θα χρησιμοποιήσω αυτή τη στήλη σαν
index γιατί τα κενά μπορεί να
δημιουργήσουν πρόβλημα.
"""
df_chart_tbl["concat"] = (df_chart_tbl["subject_id"].astype(str)
                          + df_chart_tbl["hadm_id"].astype(str)
                          + df_chart_tbl["date"].astype(str)
                          + df_chart_tbl["time"].astype(str))

"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace('-','')
df_chart_tbl['concat'] = df_chart_tbl['concat'].str.replace(':','')

In [4]:
# Ορίζω την στήλη concat ως index των γραμμών
df_chart_tbl = df_chart_tbl.set_index('concat')

In [5]:
"""
Δημιουργώ την στήλη concat και στον πίνακα
που περιλαμβάνει τις μετρήσεις για να είναι
το κοινό σημείο ανάμεσα στους δυο πίνακες
"""
df_max_chart["concat"] = (df_max_chart["subject_id"].astype(str)
                          + df_max_chart["hadm_id"].astype(str)
                          + df_max_chart["date"].astype(str)
                          + df_max_chart["time"].astype(str))

"""
Αφαιρώ τους χαρακτήρες - και :
από της τιμές της concat για να
αποφύγω πιθανά προβλήματα
"""
df_max_chart['concat'] = df_max_chart['concat'].str.replace('-','')
df_max_chart['concat'] = df_max_chart['concat'].str.replace(':','')

In [6]:
df_max_chart

,subject_id,hadm_id,itemid,Max_Chart,date,time,concat
0,10004733,27411876,220045,97.0,2174-12-04,08:00:00,100047332741187621741204080000
1,10004733,27411876,220045,107.0,2174-12-04,16:00:00,100047332741187621741204160000
2,10004733,27411876,220045,107.0,2174-12-05,00:00:00,100047332741187621741205000000
3,10004733,27411876,220045,117.0,2174-12-05,08:00:00,100047332741187621741205080000
4,10004733,27411876,220045,105.0,2174-12-05,16:00:00,100047332741187621741205160000
...,...,...,...,...,...,...,...
4177453,19999987,23865745,228412,5.0,2145-11-04,00:00:00,199999872386574521451104000000
4177454,19999987,23865745,228412,5.0,2145-11-04,08:00:00,199999872386574521451104080000
4177455,19999987,23865745,228412,5.0,2145-11-04,16:00:00,199999872386574521451104160000
4177456,19999987,23865745,228648,0.0,2145-11-03,00:00:00,199999872386574521451103000000


In [7]:
# Γεμίζω τον πίνακα
for row in df_max_chart.itertuples():
    con = row.concat
    item = row.itemid
    value = row.Max_Chart
    df_chart_tbl.at['{}'.format(con), '{}'.format(item)] = value

In [8]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τους κωδικούς των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o21_fill_chart_max_itemid_header_tbl.csv', index=False)

In [9]:
df_data_chart = df_data_chart.set_index('itemid')

In [10]:
# Ενοποίηση τιμών σε κελιά
df_data_chart["header"] = ("Max - "
                           + df_data_chart["label"].astype(str)
                           + " ("
                           + df_data_chart["unitname"].astype(str)
                           + ")")

In [11]:
"""
Ελέγχω τις κεφαλίδες και αν δεν ανήκουν στις
τέσσερις πρώτες τιμές κάνω τις αντικατάσταση.
"""
for col in df_chart_tbl.columns:
    if ((col != 'subject_id') and
        (col != 'hadm_id') and
        (col != 'date') and
        (col != 'time')):
        temp = (df_data_chart.loc[int(col), 'header'])
        df_chart_tbl.rename(columns={'{}'.format(col): '{}'.format(temp)}, inplace=True)

In [12]:
df_chart_tbl

,subject_id,hadm_id,date,time,Max - Heart Rate (bpm),Max - Respiratory Rate (insp/min),Max - O2 saturation pulseoxymetry (%),Max - Non Invasive Blood Pressure systolic (mmHg),Max - Non Invasive Blood Pressure diastolic (mmHg),Max - Non Invasive Blood Pressure mean (mmHg),...,Max - Resistance (cmH2O/L/seconds),Max - ABI Ankle BP L (mmHg),Max - ABI Brachial BP R (mmHg),Max - ITBVI (PiCCO) (mL/m2),Max - Patient/Family Informed_V1 (nan),Max - Pheresis Catheter Line Tip Cultured (nan),Max - Triple Introducer Line Tip Cultured (nan),Max - Depth at which wedge obtained (PA Line) (cm),Max - Clot Size (cm) (cm),Max - Resting Energy Expenditure (kcal/day)
concat,,,,,,,,,,,,,,,,,,,,,
100047332741187621741204080000,10004733,27411876,2174-12-04,08:00:00,97.0,22.0,100.0,176.0,112.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741204160000,10004733,27411876,2174-12-04,16:00:00,107.0,21.0,100.0,181.0,111.0,128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205000000,10004733,27411876,2174-12-05,00:00:00,107.0,29.0,98.0,168.0,84.0,101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205080000,10004733,27411876,2174-12-05,08:00:00,117.0,31.0,97.0,191.0,89.0,110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100047332741187621741205160000,10004733,27411876,2174-12-05,16:00:00,105.0,20.0,98.0,160.0,80.0,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199999872386574521451103080000,19999987,23865745,2145-11-03,08:00:00,103.0,21.0,100.0,115.0,72.0,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451103160000,19999987,23865745,2145-11-03,16:00:00,107.0,23.0,100.0,116.0,76.0,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199999872386574521451104000000,19999987,23865745,2145-11-04,00:00:00,120.0,31.0,100.0,138.0,116.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Αποθήκευση του τελικού αρχείου με επικεφαλίδες τις ονομασίες των παρατηρήσεων
df_chart_tbl.to_csv('D:\Papers\Cyprus_Paper\Data\o22_fill_max_chart_label_header_tbl.csv', index=False)